# BeautifulSoup Crawling

In [1]:
from bs4 import BeautifulSoup

In [141]:
with open('00_example.html','r',encoding='utf8') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [142]:
soup.select('p')

[<p>a</p>,
 <p>b</p>,
 <p>c</p>,
 <p>1</p>,
 <p>2</p>,
 <p>3</p>,
 <p>X</p>,
 <p>Y</p>,
 <p>Z</p>,
 <p>This is a paragraph.</p>,
 <p>This is another paragraph.</p>]

- select(): 리턴값 리스트
- select_one(): 원소개수가 하나일 때. string으로 만들려면 text로 써야 함

In [143]:
soup.select('#ex_id')

[<div id="ex_id">
 <p>X</p>
 <p>Y</p>
 <p>Z</p>
 </div>]

In [168]:
atag = soup.select_one('a') # select로 하면 리스트가 나와버리니까 안됐던 거임
atag.attrs['href']

'www.naver.com'

- 인터넷에서 가져오기

In [144]:
import requests
url = "http://200.1.220.217:3000/bbs/list/1"
res = requests.get(url)
raw = res.text
html = BeautifulSoup(raw,'html.parser')

In [170]:
rows = html.select('tr.d-flex td')
rows

[<td class="col-1" style="text-align: center;"><strong>번호</strong></td>,
 <td class="col-6" style="text-align: center;"><strong>제목</strong></td>,
 <td class="col-2" style="text-align: center;"><strong>글쓴이</strong></td>,
 <td class="col-2" style="text-align: center;"><strong>날짜/시간</strong></td>,
 <td class="col-1" style="text-align: center;"><strong>조회수</strong></td>,
 <td class="col-1" style="text-align: center;">1013</td>,
 <td class="col-6"><a href="/bbs/bid/1013/inc/1"><strong>리눅스 글쓰기</strong></a></td>,
 <td class="col-2" style="text-align: center;">홍길동</td>,
 <td class="col-2" style="text-align: center;">2020-10-27</td>,
 <td class="col-1" style="text-align: center;">3</td>,
 <td class="col-1" style="text-align: center;">1012</td>,
 <td class="col-6"><a href="/bbs/bid/1012/inc/1"><strong>마이크로소프트 엣지에서의 동작<span class="text-danger">[2]</span></strong></a></td>,
 <td class="col-2" style="text-align: center;">김은숙</td>,
 <td class="col-2" style="text-align: center;">2020-10-23</td>,
 <td

In [200]:
for row in rows[5:]:
    # if row == rows[0]:
    #    continue 이렇게 하는것보다 위에서 rows[5:]를 조절하는 게 더 좋다.
    print(row.text)

1013
리눅스 글쓰기
홍길동
2020-10-27
3
1012
마이크로소프트 엣지에서의 동작[2]
김은숙
2020-10-23
14
1011
역적 - 백성을 훔친 도적[1]
홍길동
2020-10-23
3
1010
홍길동전
홍길동
2020-10-23
3
1009
대조영[2]
대조영
2020-10-23
9
1008
Linux server를 원격 Windows에서 접속하여 글쓰기[3]
관리자
2020-10-23
7
1007
ubuntu에서 글쓰기[1]
관리자
2020-10-22
5
1006
슬기로운 의사생활[3]
이우정
2020-10-22
11
1005
파리의 연인[4]
김은숙
2020-10-22
8
1004
시크릿 가든
김은숙
2020-10-22
4


### 데이터 프레임 만들기
- 컬럼과 리스트로 이루어진 딕셔너리 만들기
- Data Preprocessing이 중요한 이유

In [242]:
bids = [] ; titles = []; replycounts = []; names = []; times =[]; view_counts=[]


trs = html.select('tr')
# print(trs)
for tr in trs[1:]:
    tds = tr.select('td') # 리스트로 나옴
    # text이든 get_text()이든 다 됨
    
    bid = tds[0].text ; bids.append(bid)
    
    span = tds[1].select_one('span')
    # 댓글수가 span태그 안에 들어가 있음
    # 즉 span태그가 있으면 댓글이 있는거고 없으면 댓글이 없음

    title = tds[1].text ; bracket = title.find('[')
    title = tds[1].text[:bracket] if span else title
    titles.append(title)

    replycount = tds[1].select_one('span.text-danger')
    replycount = 0 if replycount == None else replycount.text.strip('\[\]')
    replycounts.append(replycount)

    name = tds[2].get_text() ; names.append(name)
    time = tds[3].text ; times.append(time)
    view_count = tds[4].text ; view_counts.append(view_count)

In [244]:
import pandas as pd
bbs = pd.DataFrame({
    '번호':bids,
    '제목':titles,
    '댓글 수':replycounts,
    '글쓴이':names,
    '날짜/시간':times,
    '조회수':view_counts,
})
bbs.set_index('번호', inplace=True)
bbs

,제목,댓글 수,글쓴이,날짜/시간,조회수
번호,,,,,
1013,리눅스 글쓰기,0,홍길동,2020-10-27,3
1012,마이크로소프트 엣지에서의 동작,2,김은숙,2020-10-23,14
1011,역적 - 백성을 훔친 도적,1,홍길동,2020-10-23,3
1010,홍길동전,0,홍길동,2020-10-23,3
1009,대조영,2,대조영,2020-10-23,9
1008,Linux server를 원격 Windows에서 접속하여 글쓰기,3,관리자,2020-10-23,7
1007,ubuntu에서 글쓰기,1,관리자,2020-10-22,5
1006,슬기로운 의사생활,3,이우정,2020-10-22,11
1005,파리의 연인,4,김은숙,2020-10-22,8


- 페이지 돌아가면서 크롤링하기

In [270]:
baseurl = "http://200.1.220.217:3000/bbs/list/"
page = 2

url = baseurl + str(page)

res = requests.get(url)
raw = res.text
html = BeautifulSoup(raw,'html.parser')

In [271]:
lis = html.select('.page-item')
lis

[<li class="page-item">
 <a aria-label="Previous" class="page-link active" href="#">
 <span aria-hidden="true">«</span></a>
 </li>,
 <li class="page-item"><a class="page-link" href="/bbs/list/1">1</a></li>,
 <li aria-current="page" class="page-item active">
 <span class="page-link">
                                 2<span class="sr-only">(current)</span>
 </span>
 </li>,
 <li class="page-item">
 <a aria-label="Next" class="page-link" href="#">
 <span aria-hidden="true">»</span></a>
 </li>]

In [301]:
import re
page = lis[-2].select_one('span').text.strip('\n ')
page = int(re.sub('[()]|[a-z]+','',page))
page

2

In [303]:
for i in range(1,page+1):
    baseurl = "http://200.1.220.217:3000/bbs/list/"

    url = baseurl + str(page)

    res = requests.get(url)
    raw = res.text
    html = BeautifulSoup(raw,'html.parser')
    trs = html.select('tr')
# print(trs)
    for tr in trs[1:]:
        tds = tr.select('td') # 리스트로 나옴
        # text이든 get_text()이든 다 됨

        bid = tds[0].text ; bids.append(bid)

        span = tds[1].select_one('span')
        # 댓글수가 span태그 안에 들어가 있음
        # 즉 span태그가 있으면 댓글이 있는거고 없으면 댓글이 없음

        title = tds[1].text ; bracket = title.find('[')
        title = tds[1].text[:bracket] if span else title
        titles.append(title)

        replycount = tds[1].select_one('span.text-danger')
        replycount = 0 if replycount == None else replycount.text.strip('\[\]')
        replycounts.append(replycount)

        name = tds[2].get_text() ; names.append(name)
        time = tds[3].text ; times.append(time)
        view_count = tds[4].text ; view_counts.append(view_count)


In [304]:
bbs = pd.DataFrame({
    '번호':bids,
    '제목':titles,
    '댓글 수':replycounts,
    '글쓴이':names,
    '날짜/시간':times,
    '조회수':view_counts,
})
bbs.set_index('번호', inplace=True)
bbs

,제목,댓글 수,글쓴이,날짜/시간,조회수
번호,,,,,
1013,리눅스 글쓰기,0,홍길동,2020-10-27,3
1012,마이크로소프트 엣지에서의 동작,2,김은숙,2020-10-23,14
1011,역적 - 백성을 훔친 도적,1,홍길동,2020-10-23,3
1010,홍길동전,0,홍길동,2020-10-23,3
1009,대조영,2,대조영,2020-10-23,9
1008,Linux server를 원격 Windows에서 접속하여 글쓰기,3,관리자,2020-10-23,7
1007,ubuntu에서 글쓰기,1,관리자,2020-10-22,5
1006,슬기로운 의사생활,3,이우정,2020-10-22,11
1005,파리의 연인,4,김은숙,2020-10-22,8
